In [1]:
from openai import OpenAI
from typing import List, Dict
import itertools
import time
from tqdm import tqdm
import evaluate
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Initialize BLEU and BERTScore metrics
bleu_metric = evaluate.load("bleu")
bertscore_metric = evaluate.load("bertscore")

class LegalDeepseekAnswerer:
    def __init__(self, api_key, model="deepseek-ai/deepseek-r1-distill-qwen-7b"):
        self.client = OpenAI(
            api_key=api_key,
            base_url="https://integrate.api.nvidia.com/v1"
        )
        self.model = model

    def generate(self, prompt, temperature=0.3, top_p=1.0, best_of_n=1, debug=False, retries=3, wait_time=10):
        messages = [{"role": "user", "content": prompt}]
        candidates = []

        for _ in range(best_of_n):
            for attempt in range(retries):
                try:
                    completion = self.client.chat.completions.create(
                        model=self.model,
                        messages=messages,
                        temperature=temperature,
                        top_p=top_p,
                        max_tokens=1024,
                        stream=False
                    )
                    response_text = completion.choices[0].message.content.strip()

                    if debug:
                        print("🧠 Raw Response:\n", response_text)

                    candidates.append(response_text)
                    break
                except Exception as e:
                    print(f"⚠️ Attempt {attempt+1} failed: {e}")
                    if attempt < retries - 1:
                        print(f"⏳ Retrying in {wait_time}s...")
                        time.sleep(wait_time)
                    else:
                        raise RuntimeError("❌ All retries failed for Deepseek")

        return candidates

def simple_reranker(candidates: List[str]) -> str:
    return max(candidates, key=lambda x: len(x))

def evaluate_outputs(predictions: List[str], references: List[str]) -> Dict:
    bleu = bleu_metric.compute(predictions=predictions,
                               references=[[ref] for ref in references])['bleu']
    bert = bertscore_metric.compute(predictions=predictions, references=references, lang='en')['f1']
    bert_avg = sum(bert) / len(bert)
    return {"BLEU": bleu, "BERTScore": bert_avg}

def hyperparameter_grid_search(answerer: LegalDeepseekAnswerer,
                                prompts: List[str],
                                references: List[str],
                                temperatures: List[float],
                                top_ps: List[float],
                                best_of_n: int = 3):
    all_results = []
    grid = list(itertools.product(temperatures, top_ps))

    for temp, top_p in tqdm(grid, desc="Grid Search"):
        batch_predictions = []
        for prompt in prompts:
            candidates = answerer.generate(
                prompt,
                temperature=temp,
                top_p=top_p,
                best_of_n=best_of_n
            )
            best_answer = simple_reranker(candidates)
            batch_predictions.append(best_answer)
        scores = evaluate_outputs(batch_predictions, references)
        result = {
            "temperature": temp,
            "top_p": top_p,
            "BLEU": scores["BLEU"],
            "BERTScore": scores["BERTScore"]
        }
        print(f"Config Tested: {result}")
        all_results.append(result)

    return all_results

def log_results_and_plot(results: List[Dict], csv_path: str, heatmap_path: str):
    df = pd.DataFrame(results)
    df.to_csv(csv_path, index=False)
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    pivot_bleu = df.pivot(index="temperature", columns="top_p", values="BLEU")
    sns.heatmap(pivot_bleu, annot=True, fmt=".5f", cmap="YlGnBu")
    plt.title("BLEU Score Heatmap")

    plt.subplot(1, 2, 2)
    pivot_bert = df.pivot(index="temperature", columns="top_p", values="BERTScore")
    sns.heatmap(pivot_bert, annot=True, fmt=".5f", cmap="YlOrRd")
    plt.title("BERTScore Heatmap")

    plt.tight_layout()
    plt.savefig(heatmap_path, dpi=300)
    plt.close()
    print(f"\nResults saved to {csv_path} and heatmap to {heatmap_path}")

# =========================
# Example Usage
# =========================
if __name__ == "__main__":
    API_KEY = "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"

    answerer = LegalDeepseekAnswerer(api_key=API_KEY)

    prompts = [
        "Question: What is the principle of stare decisis in US law? Answer and give reasoning.",
        "Question: Can a minor legally enter into a binding contract? Explain with reasoning."
    ]
    references = [
        "Answer: The principle of stare decisis means that courts follow precedents established by higher courts. Reasoning: This ensures legal consistency and predictability.",
        "Answer: Generally, a minor cannot enter into a binding contract, except for necessities. Reasoning: Contracts with minors are usually voidable to protect them from exploitation."
    ]

    temperatures = [0.1, 0.3, 0.5, 0.7]
    top_ps = [0.5, 0.7, 0.9]

    results = hyperparameter_grid_search(
        answerer=answerer,
        prompts=prompts,
        references=references,
        temperatures=temperatures,
        top_ps=top_ps,
        best_of_n=3
    )

    log_results_and_plot(
        results,
        csv_path="deepseek_hyperparameter_tuning.csv",
        heatmap_path="deepseek_hyperparameter_heatmap.png"
    )

Grid Search:   0%|                                       | 0/12 [00:00<?, ?it/s]Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Grid Search:   8%|██▌                            | 1/12 [01:21<14:53, 81.19s/it]

Config Tested: {'temperature': 0.1, 'top_p': 0.5, 'BLEU': 0.0042028326736153225, 'BERTScore': 0.8186533153057098}


Grid Search:  17%|█████▏                         | 2/12 [02:43<13:38, 81.86s/it]

Config Tested: {'temperature': 0.1, 'top_p': 0.7, 'BLEU': 0.003888413256618598, 'BERTScore': 0.8186072409152985}


Grid Search:  25%|███████▊                       | 3/12 [03:56<11:39, 77.67s/it]

...

Grid Search:  92%|███████████████████████████▌  | 11/12 [14:22<01:19, 79.68s/it]

Config Tested: {'temperature': 0.7, 'top_p': 0.7, 'BLEU': 0.004939112373487706, 'BERTScore': 0.8198120594024658}


Grid Search: 100%|██████████████████████████████| 12/12 [15:43<00:00, 78.60s/it]

Config Tested: {'temperature': 0.7, 'top_p': 0.9, 'BLEU': 0.004144798123808805, 'BERTScore': 0.8208663463592529}



Results saved to deepseek_hyperparameter_tuning.csv and heatmap to deepseek_hyperparameter_heatmap.png
